In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import regex as re
import misfunciones as mf

In [2]:
sharks = pd.read_csv("data/attacks.csv",encoding = "ISO-8859-1")

In [3]:
#sharks

In [4]:
#limpiamos todas las columnas y filas que 
sharks.dropna(how="all", inplace=True)
sharks.dropna(how="all",axis = 1, inplace=True)

# Hipótesis
- hipotesis 1 los ataques de tiburones han icrmentado según han ido avanzando los años
- hipótesis 2 los ataquesde tiburones atacan más a embarcaciones que a nadadores
- hipóstesis 3 Florida es la capital mundial de los ataques de tiburones:
        https://www.lavanguardia.com/ocio/viajes/20210407/6631447/6-playas-mas-peligrosas-mundo.html
- hipótesis 4 ¿Es posible el ataque de tiburón en la costa española?
        https://www.mundo-geo.es/naturaleza/es-posible-ataque-tiburon-en-costa-espanola_238643_102.html

In [5]:
columnas = list(sharks.columns)
print(columnas)

['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location', 'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time', 'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href', 'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22', 'Unnamed: 23']


In [6]:
#columnas que no necesito para ninguna de mis hipótesis (primera pasasda)
columnas_no = ['Case Number', #no
'Investigator or Source', #no
 'pdf', #no
 'href formula', #no
 'href', #no
 'Case Number.1', #no
 'Case Number.2', #no
 'original order', #no
 'Unnamed: 22',#no
 'Unnamed: 23'] #no

sharks.drop(columnas_no, axis=1, inplace=True)

In [7]:
#ahora que hemos borrado columnas vemos si quedan filas que son nan por completo y borrranos
sharks.dropna(how="all", inplace=True)

In [8]:
columnas = sharks.columns
columnas

Index(['Date', 'Year', 'Type', 'Country', 'Area', 'Location', 'Activity',
       'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time', 'Species '],
      dtype='object')

In [9]:
#areas = pd.DataFrame(sharks.Area.value_counts())
#areas

### Limpieza de datos nulos

In [10]:
sharks.isna().sum() #miramos cuantos datos nulos hay en cda columna.

Date              0
Year              2
Type              4
Country          50
Area            455
Location        540
Activity        544
Name            210
Sex             565
Age            2831
Injury           28
Fatal (Y/N)     539
Time           3354
Species        2838
dtype: int64

In [11]:
sharks.Date.sample(20) #limpiar fechas por día, mes y año.
#quitar reported, circa
sharks.Date.value_counts()

1957                          11
1942                           9
1956                           8
1958                           7
1950                           7
                              ..
01-Dec-1994                    1
Reported      10-Dec-1994      1
11-Dec-1994                    1
13-Dec-1994                    1
1845-1853                      1
Name: Date, Length: 5433, dtype: int64

In [12]:
sharks.Type.unique() #solo hay 4 nulos

array(['Boating', 'Unprovoked', 'Invalid', 'Provoked', 'Questionable',
       'Sea Disaster', nan, 'Boat', 'Boatomg'], dtype=object)

In [13]:
sharks[sharks.Type == "Invalid"].sample(5) #miramos a ver si los datos nulos pueden sustitirse por "Ivalid"

,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species
5951,1864,1864.0,Invalid,AUSTRALIA,South Australia,"Corio Bay, Port Phillip",Swimming,"Mr. Warren, Jr.",M,NaN,Presumed FATAL,NaN,NaN,Shark involvement prior to death unconfirmed
5989,17-Aug-1858,1858.0,Invalid,USA,New York,"Staten Island, Richmond County",Swimming,male,M,NaN,Thought to have been taken by a shark/s. Body ...,NaN,NaN,Shark involvement prior to death unconfirmed
2658,14-Oct-1989,1989.0,Invalid,USA,Hawaii,"Kahe Point, O'ahu",Scuba diving,"Ray Mehl, Jr.",M,NaN,Disappeared 15 minutes into shallow dive. Deca...,NaN,16h30,Shark involvement suspected but not confirmed
4781,1940,1940.0,Invalid,SOUTH AFRICA,Eastern Cape Province,"Kowie River Mouth, Port Alfred",Standing in water with child in her arms,female,F,NaN,Leg bitten,NaN,NaN,Questionable incident
3555,Sep- 1966,1966.0,Invalid,BAHAMAS,Bimini Islands,Tongue of the Ocean,Diving / UW photography,Jerry Greenberg,M,NaN,2.4 to 3 m [8' to 10'] oceanic whitetip shark ...,NaN,NaN,Invalid


In [14]:
sharks.Type.fillna("Invalid",inplace = True)

In [15]:
sharks["Fatal (Y/N)"].unique()

array(['N', 'Y', nan, 'M', 'UNKNOWN', '2017', ' N', 'N ', 'y'],
      dtype=object)

In [16]:
sharks["Fatal (Y/N)"].fillna("UNKNOWN",inplace = True)
sharks["Fatal (Y/N)"].unique() #hay que luego limpiar los datos para que no haya espacios y queden solo N,Y,UNKOWN

array(['N', 'Y', 'UNKNOWN', 'M', '2017', ' N', 'N ', 'y'], dtype=object)

In [17]:
sharks["Injury"].unique()

array(['No injury to occupant, outrigger canoe and paddle damaged',
       'Minor injury to left thigh',
       'Injury to left lower leg from surfboard skeg', ...,
       'FATAL, leg stripped of flesh  ',
       'FATAL, knocked overboard by tail of shark & carried off by shark ',
       'FATAL. "Shark bit him in half, carrying away the lower extremities" '],
      dtype=object)

In [18]:
sharks["Injury"].fillna("UNKNOWN",inplace = True) #rellenamos también injury con UNKOWN para los nan 
#quizas quite esta columna más tarde

In [19]:
sharks["Name"].unique()
sharks["Name"].fillna("John/Jane Doe",inplace = True)
#puede que la quitemos, porque con saber el sexo de la persona nos sobra

In [20]:
#sharks["Species"].replace(".*[wW](hite|HITE).*","White",inplace = True)

In [21]:
#cambiamos year a enteros
sharks["Year"].fillna(0.0,inplace = True)
sharks["Year"] = sharks["Year"].apply(mf.entero)

In [22]:
sharks["Sex "].unique()
sharks["Sex "].fillna("X", inplace = True)
#luego cambiar nombre columna y cambiar datos que no sean M, F, o X para desconocido.

In [23]:
actividades = sharks["Activity"].value_counts() #con esto se hace un objeto que nos puede servir de diccionario

In [24]:
sharks["Country"].unique()
sharks["Country"].fillna("UNKNOWN", inplace = True)

In [25]:
sharks["Age"].fillna("UNKNOWN", inplace = True)

In [26]:
sharks["Area"].fillna("UNKNOWN", inplace = True)
sharks["Location"].fillna("UNKNOWN", inplace = True)
sharks["Activity"].fillna("UNKNOWN", inplace = True)
sharks["Species "].fillna("UNKNOWN", inplace = True) #cambiar nombre de la columna a "Species""
sharks["Time"].fillna("UNKNOWN", inplace = True)

In [27]:
#cambio nombre de algunas columnas que tienen espacios o caracteres raros en el nombre
list(columnas) #"Sex " "Species " Fatal (Y/N)
cols = ["Sex ","Species ","Fatal (Y/N)"]
new_names = {"Sex " : "Sex",
                 "Species " : "Species",
                 "Fatal (Y/N)" : "Fatal"}
sharks.rename(columns = new_names, inplace=True)
sharks.columns

Index(['Date', 'Year', 'Type', 'Country', 'Area', 'Location', 'Activity',
       'Name', 'Sex', 'Age', 'Injury', 'Fatal', 'Time', 'Species'],
      dtype='object')

In [28]:
sharks["Sex"].unique() #'M ', 'lli', 'N', '.' -- X = desconocido
sharks["Sex"].replace('M ',"M",inplace=True)
sharks["Sex"].replace('lli',"X",inplace=True)
sharks["Sex"].replace('.',"X",inplace=True)
sharks["Sex"].replace('N',"X",inplace=True)

In [29]:
sharks["Sex"].unique()

array(['F', 'M', 'X'], dtype=object)

In [30]:
sharks[(sharks.Country == "UNKNOWN" )&(sharks.Area == "UNKNOWN" )&(sharks.Location == "UNKNOWN" )] #29 líneas ¿las borro?

,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal,Time,Species
62,Reported 13-Nov-2017,2017,Unprovoked,UNKNOWN,UNKNOWN,UNKNOWN,Surfing,Timur Yunusov,M,24,Puncture wounds to feet,N,UNKNOWN,UNKNOWN
525,Aug-2014,2014,Invalid,UNKNOWN,UNKNOWN,UNKNOWN,Sea disaster,Cuban refugees,M,UNKNOWN,Shark involvement prior to death not confirmed,UNKNOWN,UNKNOWN,Shark involvement not confirmed
3378,Nov-1970,1970,Unprovoked,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,Heinz Plotsky,M,UNKNOWN,Extensive injuries,N,UNKNOWN,UNKNOWN
3388,5-Jul-1970,1970,Unprovoked,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,male,M,UNKNOWN,Finger or toe severed,N,Night,Mako shark
3395,Apr-1970,1970,Provoked,UNKNOWN,UNKNOWN,UNKNOWN,Freediving,Lionel Jarvis,M,UNKNOWN,Arm abraded & lacerated. Recorded as PROVOKED ...,N,UNKNOWN,Wobbegong shark
3399,5-Feb-1970,1970,Unprovoked,UNKNOWN,UNKNOWN,UNKNOWN,Wading,Sally Anne Irvine,F,8,Lacerations to lower leg,N,UNKNOWN,Carpet shark
3425,Aug-1969,1969,Unprovoked,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,Rodney Hughes,M,25,Am lacerated,N,UNKNOWN,UNKNOWN
3661,27-Sep-1964,1964,Invalid,UNKNOWN,UNKNOWN,UNKNOWN,Spearfishing,Giancarlo Griffon,M,24,"Disappeared, probable drowning but sharks in a...",UNKNOWN,11h00,Shark involvement prior to death was not confi...
4726,1942,1942,Sea Disaster,UNKNOWN,UNKNOWN,UNKNOWN,Jumped overboard from torpedoed Panamanian fre...,male,M,UNKNOWN,FATAL,Y,UNKNOWN,UNKNOWN
4729,1942,1942,Boating,UNKNOWN,UNKNOWN,UNKNOWN,"Days before the surrender of Singapore, the 3 ...","Bombardier J. Hall, Private Green of the Sherw...",M,UNKNOWN,No injury to occupants. Sharks continually fol...,N,UNKNOWN,UNKNOWN


In [31]:
list(sharks["Date"].unique())
#eliminar "Reported "
#[rR](eported|EPORTED)\s\d{2}.[A-Z][a-z][a-z].\d{4}"
sharks["Date"] = sharks["Date"].str.extract(r'(\d{2}.[A-Z][a-z][a-z].\d{4})')


In [32]:
sharks["Date"].fillna("00-000-0000", inplace = True)

In [33]:
sharks["Month"] = sharks["Date"].str.extract(r'([A-Z][a-z]{2})')

In [34]:
sharks["Month"].fillna("000",inplace = True)

In [35]:
lista = list(sharks["Year"][(sharks.Year < 1543)&(sharks.Year >0)])
lista

[500, 77, 5]

In [36]:
#cambiamos esos años que son menores de 1543 por 0
#recordar que 0 es "año desconocido"
sharks["Year"] = sharks.Year.apply(lambda yea:(mf.cambia(0)) if yea in lista else yea ) 

In [37]:
sharks.Year.unique()

array([2018, 2017,    0, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009,
       2008, 2007, 2006, 2005, 2004, 2003, 2002, 2001, 2000, 1999, 1998,
       1997, 1996, 1995, 1984, 1994, 1993, 1992, 1991, 1990, 1989, 1969,
       1988, 1987, 1986, 1985, 1983, 1982, 1981, 1980, 1979, 1978, 1977,
       1976, 1975, 1974, 1973, 1972, 1971, 1970, 1968, 1967, 1966, 1965,
       1964, 1963, 1962, 1961, 1960, 1959, 1958, 1957, 1956, 1955, 1954,
       1953, 1952, 1951, 1950, 1949, 1948, 1848, 1947, 1946, 1945, 1944,
       1943, 1942, 1941, 1940, 1939, 1938, 1937, 1936, 1935, 1934, 1933,
       1932, 1931, 1930, 1929, 1928, 1927, 1926, 1925, 1924, 1923, 1922,
       1921, 1920, 1919, 1918, 1917, 1916, 1915, 1914, 1913, 1912, 1911,
       1910, 1909, 1908, 1907, 1906, 1905, 1904, 1903, 1902, 1901, 1900,
       1899, 1898, 1897, 1896, 1895, 1894, 1893, 1892, 1891, 1890, 1889,
       1888, 1887, 1886, 1885, 1884, 1883, 1882, 1881, 1880, 1879, 1878,
       1877, 1876, 1875, 1874, 1873, 1872, 1871, 18

In [38]:
sharks = sharks[sharks["Year"] > 1800]

In [39]:
sharks.Age.unique()

array(['57', '11', '48', 'UNKNOWN', '18', '52', '15', '12', '32', '10',
       '21', '34', '30', '60', '33', '29', '54', '41', '37', '56', '19',
       '25', '69', '38', '55', '35', '46', '45', '14', '40s', '28', '20',
       '24', '26', '49', '22', '7', '31', '17', '40', '13', '42', '3',
       '8', '50', '16', '82', '73', '20s', '68', '51', '39', '58', 'Teen',
       '47', '61', '65', '36', '66', '43', '60s', '9', '72', '59', '6',
       '27', '64', '23', '71', '44', '62', '63', '70', '18 months', '53',
       '30s', '50s', 'teen', '77', '74', '28 & 26', '5', '86', '18 or 20',
       '12 or 13', '46 & 34', '28, 23 & 30', 'Teens', '36 & 26',
       '8 or 10', '84', '\xa0 ', ' ', '30 or 36', '6½', '21 & ?', '75',
       '33 or 37', 'mid-30s', '23 & 20', ' 30', '7      &    31', ' 28',
       '20?', "60's", '32 & 30', '16 to 18', '87', '67', 'Elderly',
       'mid-20s', 'Ca. 33', '74 ', '45 ', '21 or 26', '20 ', '>50',
       '18 to 22', 'adult', '9 & 12', '? & 19', '9 months', '25 to 3

In [40]:

#borrar los espacios detrás de los números
#cambiar lso strings si son numéricos
sharks["Age"] = sharks["Age"].str.strip()
sharks["Age"] = sharks["Age"].apply(mf.entero_if)




In [41]:
sharks.Age.unique()

array([57, 11, 48, 'UNKNOWN', 18, 52, 15, 12, 32, 10, 21, 34, 30, 60, 33,
       29, 54, 41, 37, 56, 19, 25, 69, 38, 55, 35, 46, 45, 14, '40s', 28,
       20, 24, 26, 49, 22, 7, 31, 17, 40, 13, 42, 3, 8, 50, 16, 82, 73,
       '20s', 68, 51, 39, 58, 'Teen', 47, 61, 65, 36, 66, 43, '60s', 9,
       72, 59, 6, 27, 64, 23, 71, 44, 62, 63, 70, '18 months', 53, '30s',
       '50s', 'teen', 77, 74, '28 & 26', 5, 86, '18 or 20', '12 or 13',
       '46 & 34', '28, 23 & 30', 'Teens', '36 & 26', '8 or 10', 84, '',
       '30 or 36', '6½', '21 & ?', 75, '33 or 37', 'mid-30s', '23 & 20',
       '7      &    31', '20?', "60's", '32 & 30', '16 to 18', 87, 67,
       'Elderly', 'mid-20s', 'Ca. 33', '21 or 26', '>50', '18 to 22',
       'adult', '9 & 12', '? & 19', '9 months', '25 to 35', '23 & 26', 1,
       '(adult)', '33 & 37', '25 or 28', '37, 67, 35, 27,  ? & 27',
       '21, 34,24 & 35', '30 & 32', '50 & 30', '17 & 35', 'X',
       '"middle-age"', '13 or 18', '34 & 19', '33 & 26', '2 to 3 months

In [42]:
len(list(sharks.Injury.unique()))

3670

In [43]:
#sharks["Species"].replace(".*[wW](hite|HITE).*","White",inplace = True)

sharks["Injury"] = sharks["Injury"].str.replace(".*[Nn][Oo]\s[Ii](njur|NJUR).*","No injury",regex = True)
sharks["Injury"] = sharks["Injury"].str.replace(".*[Nn][Oo]\s[Ii](nur|NUR).*","No injury",regex = True)
sharks["Injury"] = sharks["Injury"].str.replace(".*[Mm](INOR|inor).*","Minor injury",regex = True)
sharks["Injury"] = sharks["Injury"].str.replace(".*[Ss](ERIOUS|erious).*","Serious injury",regex = True)
sharks["Injury"] = sharks["Injury"].str.replace(".*[Ff](ATAL|atal).*","Fatal",regex = True)
sharks["Injury"] = sharks["Injury"].str.replace(".*[Ll](ACERAT|acerat).*","Lacerations",regex = True)
sharks["Injury"] = sharks["Injury"].str.replace(".*[Ii](njur|NJUR).*","Injured",regex = True)
sharks["Injury"] = sharks["Injury"].str.replace(".*[Ii](nur|NUR).*","Injured",regex = True)
sharks["Injury"] = sharks["Injury"].str.replace(".*[Bb](IT|it).*","Bitten",regex = True)
sharks["Injury"] = sharks["Injury"].str.replace(".*[Pp](UNCTUR|unctur).*","Punctured wounds",regex = True)
sharks["Injury"] = sharks["Injury"].str.replace(".*[Aa](brasi|BRASI).*","Abrasions",regex = True)
sharks["Injury"] = sharks["Injury"].str.replace(".*[Ss](EVERED|evered).*","Limb(s) severed",regex = True)
sharks["Injury"] = sharks["Injury"].str.replace(".*[Pp](ROBABLE|robable)\s[Dd](ROWN|rown).*","Probable drowning",regex = True)
sharks["Injury"] = sharks["Injury"].str.replace(".*[Dd](ROWN|rown).*","Drowned",regex = True)
sharks["Injury"] = sharks["Injury"].str.replace(".*[Ss](caven|CAVEN).*","Shark scavenged",regex = True)
sharks["Injury"] = sharks["Injury"].str.replace(".*[Pr](ROVOKED|rovoked)\s[Ii](NCIDENT|ncident).*","Drowned",regex = True)
sharks["Injury"] = sharks["Injury"].str.replace(".*[Cc](UT|ut).*","Cut(s)",regex = True)
sharks["Injury"] = sharks["Injury"].str.replace(".*[Nn][Oo]\s[Aa](T+ACK|t+ack).*","No attack",regex = True)
sharks["Injury"] = sharks["Injury"].str.replace(".*[Nn](IP+|ip+).*","nipped",regex = True)
sharks["Injury"] = sharks["Injury"].str.replace(".*[Ww](OUND|ound).*","Wounds",regex = True)
sharks["Injury"] = sharks["Injury"].str.replace(".*[Nn](OT|ot)\s[Cc](ONFIRM|onfirm),*","not confirmed",regex = True)
sharks["Injury"] = sharks["Injury"].str.replace(".*[Hh](OAX|oax).*","HOAX",regex = True)
sharks["Injury"] = sharks["Injury"].str.replace(".*[Uu](NCONFIRM|nconfirm).*","not confirmed",regex = True)
sharks["Injury"] = sharks["Injury"].str.replace(".*[Dd](oubt|OUBT).*","not confirmed",regex = True)
sharks["Injury"] = sharks["Injury"].str.replace(".*[Nn][Oo]\s[Ii](nur|NUR).*","No injury",regex = True)
sharks["Injury"] = sharks["Injury"].str.replace(".*[Rr](emai|EMAI).*","human remains found",regex = True)
sharks["Injury"] = sharks["Injury"].str.replace(".*[Bb](ruis|RUIS).*","Bruised",regex = True)
sharks["Injury"] = sharks["Injury"].str.replace(".*[Dd](isap+|ISAP+).*","Disappeared",regex = True)
sharks["Injury"] = sharks["Injury"].str.replace(".*[Mm](is+in|IS+IN).*","Disappeared",regex = True)
sharks["Injury"] = sharks["Injury"].str.replace(".*[Nn][Oo]\s[Dd](ETAIL|etails).*","UNKNOWN",regex = True)
sharks["Injury"] = sharks["Injury"].str.replace(".*[Uu](nknow|nKNOW).*","UNKNOWN",regex = True)
#Missing

In [44]:
len(list(sharks.Injury.unique()))
list(sharks.Injury.unique())


['Injured',
 'Lacerations',
 'Fatal',
 'Bitten',
 'Cut(s)',
 'Wounds',
 'Abrasions',
 'Drowned',
 'Fingernail pulled off ',
 'Limb(s) severed',
 'nipped',
 'Struck by fin on chest & leg',
 'Bruised',
 'Right hand & part of forearm removed',
 'Arm amputated below shoulder',
 'Board snapped in two',
 'HOAX',
 'not confirmeded',
 'His boat was holed by a shark',
 'UNKNOWN',
 'Scrape to right forearm. Frennet says inflicted by a blue shark, but authorities question shark involvement',
 '"Light scratch on hand/wrist area"',
 'Disappeared',
 'No ijnury to boardrider, shark struck board breaking the mast',
 'not confirmed',
 'Leg struck. Initally reported as a shark attack, but involved a swordfish',
 'Killed by a shark or crocodile.',
 'Torn shoulder ligament as result of collision with shark',
 'Erroneously reported on several internet sites as a "shark attack", it was the shark 8\', 300-kg mako shark that was attacked, not the fisherman',
 'Body not recovered',
 'Shark scavenged',
 'Report

In [45]:
sharks.Country

0             USA
1             USA
2             USA
3       AUSTRALIA
4          MEXICO
          ...    
6127    AUSTRALIA
6128          USA
6129    AUSTRALIA
6130        INDIA
6131      UNKNOWN
Name: Country, Length: 6130, dtype: object

In [46]:
(sharks.columns)

Index(['Date', 'Year', 'Type', 'Country', 'Area', 'Location', 'Activity',
       'Name', 'Sex', 'Age', 'Injury', 'Fatal', 'Time', 'Species', 'Month'],
      dtype='object')

In [47]:
list(sharks.Activity.unique())

['Paddling',
 'Standing',
 'Surfing',
 'Free diving',
 'Kite surfing',
 'Swimming',
 'Fishing',
 'Walking',
 'Feeding sharks',
 'Boogie boarding',
 'Scuba diving',
 'Paddle-skiing',
 'Body boarding',
 'Windsurfing',
 'Stand-Up Paddleboarding',
 'Wading',
 'Scuba Diving',
 'Kayak fishing for sharks',
 'Cleaning fish',
 'Snorkeling',
 'Spearfishing',
 'UNKNOWN',
 'Diving',
 'Fishing / Wading',
 '2 boats capsized',
 'Night bathing',
 'Surfing ',
 'Kayaking / Fishing',
 'Kayaking',
 'Body surfing',
 'Swimming, poaching abalone',
 'Canoeing',
 'SUP',
 'Skimboarding',
 'Touching a shark',
 'Fishing ',
 'Attempting to lasso a shark',
 'Photo shoot',
 'Paddle boarding',
 'Kakaying',
 'Washing hands',
 'Grabbing shark for a selfie',
 'Kayak fishing',
 'Tagging sharks',
 'Surf skiing ',
 'Surf fishing',
 'Floating',
 'Cage Diving',
 'Surfng',
 'SUP Foil boarding',
 'Lobstering',
 'Fishing in Alabama Deep Fishing Rodeo',
 'Fishing for squid',
 'Fishing for sharks',
 'Floating in tube',
 'Teasing 

In [48]:
#1533
sharks["Activity"] = sharks["Activity"].str.strip()
sharks["Activity"] = sharks["Activity"].str.replace(".*[Ff](ishin|ISHIN).*","Fishing",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace("[Ss](urfin|URFIN).*","Surfing",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Ff](allin|ALLIN).*","Fell into the watter",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace("[Dd](ivin|IVIN).*","Diving",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Pp](oachin|OACHIN).*","Poaching",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Pp](hoto|HOTO).*","Taking Photos",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Ww](RECK|reck)\s(OF|of)\s(THE|the).*","shipwreck",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Ss](hipwreck|HIPWRECK).*","shipwreck",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Oo](verboar|VERBOAR).*","Jump or fell overboard",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Cc](apsi|APSI).*","capsized",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Hh](UNTIN|untin).*","Hunting",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Ss](elfie|ELFIE).*","Taking Photos",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Pp](LAYIN|layin).*","Playing",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Ff](ILM|ilm).*","Filming",regex = True)
sharks["Activity"] = sharks["Activity"].str.replace(".*[Ss](ink|INK).*","Sinking",regex = True)

#Sinking 

In [49]:
list(sharks.Activity.unique())

['Paddling',
 'Standing',
 'Surfing',
 'Free Diving',
 'Kite Surfing',
 'Swimming',
 'Fishing',
 'Walking',
 'Feeding sharks',
 'Boogie boarding',
 'Scuba Diving',
 'Paddle-skiing',
 'Body boarding',
 'WindSurfing',
 'Stand-Up Paddleboarding',
 'Wading',
 'Cleaning fish',
 'Snorkeling',
 'UNKNOWN',
 'Diving',
 'capsized',
 'Night bathing',
 'Kayaking',
 'Body Surfing',
 'Poaching',
 'Canoeing',
 'SUP',
 'Skimboarding',
 'Touching a shark',
 'Attempting to lasso a shark',
 'Taking Photos',
 'Paddle boarding',
 'Kakaying',
 'Washing hands',
 'Tagging sharks',
 'Surf skiing',
 'Floating',
 'Cage Diving',
 'Surfng',
 'SUP Foil boarding',
 'Lobstering',
 'Floating in tube',
 'Teasing a shark',
 'Surf-skiing',
 'Feeding stingrays?',
 'Sea disaster',
 'Swimming / Wading',
 'Scallop Diving',
 'Treading water',
 'Playing',
 'Body Boarding',
 'Attempting to rescue a shark',
 'Standing / Snorkeling',
 'Hand feeding sharks',
 'Sitting in the water',
 'Transatlantic Rowing',
 'BodySurfing',
 'Rowin

In [54]:
len(list(sharks.Country.unique())) #202


202

In [56]:
sharks["Country"] = sharks["Country"].str.strip()
sharks.Country.unique()
len(list(sharks.Country.unique())) #194

194

In [62]:
#sharks["Date"].str.extract(r'([A-Z][a-z]{2})')
#sharks = sharks[sharks["Year"] > 1800]
#sharks[sharks["Country"] == sharks.Country.str.extract(r'(.*(UNITED|united).*)')]    


<ipython-input-62-948983d07888>:3: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version.  Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`
  sharks[sharks["Country"] == sharks.Country.str.extract(r'(.*(UNITED|united).*)')]


,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal,Time,Species,Month
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6128,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6129,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
